# 1. Data Wrangling <a id="data_wrangling"></a>

<a id="contents"></a>
# Table of Contents  
1. [Data Wrangling](#data_wrangling)
    - [1.1 Introduction](#introduction)
    - [1.2 Imports](#imports)
    - [1.3 Load Datasets](#load)
    - [1.4 Dataset Cleaning](#cleaning)

## 1.1 Introduction<a id="introduction"></a>

### Problem
Real estate investors need to identify profitable investment opportunities in dynamic markets. Understanding market trends and segmenting opportunities based on risk and return profiles is crucial for optimizing investment strategies. The goal of this project is to maximize investment returns by leveraging data-driven approaches to identify undervalued properties, forecast market trends, and optimize portfolio allocations.


### Clients
The findings of this study will be of interest to a broad range of stakeholders, specifically real estate investors, portfolio managers, and real estate agents and brokers who can benefit from understanding market trends and leverage the insights from the project to provide more accurate and data-driven recommendations.


### Data
For this project, we have identified multiple data sources to ensure we have approprirate features and targets for our model. 

Data Sources:
1) **FHFA House Price Index (HPI):** This is a broad measure of the movement of single-family house prices. The HPI is a weighted, repeat-sales index, meaning that it measures average price changes in repeat sales or refinancings on the same properties. This information is obtained by reviewing repeat mortgage transactions on single-family properties whose mortgages have been purchased or securitized by Fannie Mae or Freddie Mac since January 1975. The data file coveres dates from 1991 through 2014. 
(Link to dataset: https://www.fhfa.gov/data/hpi/datasets) 
2) **Zillow Home Value Index (ZHVI):** A measure of the typical home value and market changes across the US by housing type. It reflects the typical value for homes in the 35th to 65th percentile range. 
3) **Zillow Observed Rent Index (ZORI):** ZORI is a repeat-rent index that is weighted to the rental housing stock to ensure representativeness across the entire market, not just those homes currently listed for-rent. The index is dollar-denominated by computing the mean of listed rents that fall into the 35th to 65th percentile range for all homes and apartments across the US, which is weighted to reflect the rental housing stock.
4) **Zillow Observed Renter Demand Index (ZORDI):** ZORDI tracks engagement on Zillow’s rental listings to proxy changes in rental demand. 

Link to Zillow datasets: https://www.zillow.com/research/data/


## 1.2 Imports

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import csv
from tqdm.notebook import tqdm
from datetime import datetime, timezone

## 1.3 Load Datasets<a id="load"></a>

To begin, we are focusing the data on the regions wihthin New York State (NY) in the US. After loading all the datasets in to the notebook, we will filter out any region/location that is not in NY.

In [2]:
# Folder path where the CSV files are located
folder_path = '/Users/heatheradler/Documents/GitHub/Springboard/Springboard_Projects/Capstone 3/Datasets'

# List all files in the folder
file_list = os.listdir(folder_path)

# Filter only CSV files
csv_files = [file for file in file_list if file.endswith('.csv')]

# Dictionary to store DataFrames
dataframes = {}

# Load each CSV file into a DataFrame
for file in csv_files:
    # Construct the full file path
    file_path = os.path.join(folder_path, file)
    # Read the CSV file into a DataFrame
    df_name = file.split('.')[0]  # Use file name without extension as DataFrame name
    dataframes[df_name] = pd.read_csv(file_path)

# Example usage:
# Access each DataFrame by its name in the 'dataframes' dictionary
hpi_df = dataframes['HPI']
zori_df = dataframes['ZORI']
zhvi_df = dataframes['ZHVI']
zordi_df = dataframes['ZORDI']

# Example: Print first few rows of each DataFrame
print("First few rows of HPI dataset:")
print(hpi_df.head())

print("\nFirst few rows of ZORI dataset:")
print(zori_df.head())

print("\nFirst few rows of ZHVI dataset:")
print(zhvi_df.head())

print("\nFirst few rows of ZORDI dataset:")
print(zordi_df.head())

First few rows of HPI dataset:
      hpi_type     hpi_flavor frequency                   level  \
0  traditional  purchase-only   monthly  USA or Census Division   
1  traditional  purchase-only   monthly  USA or Census Division   
2  traditional  purchase-only   monthly  USA or Census Division   
3  traditional  purchase-only   monthly  USA or Census Division   
4  traditional  purchase-only   monthly  USA or Census Division   

                    place_name place_id    yr  period  index_nsa  index_sa  
0  East North Central Division   DV_ENC  1991       1     100.00    100.00  
1  East North Central Division   DV_ENC  1991       2     100.89    100.94  
2  East North Central Division   DV_ENC  1991       3     101.30    100.91  
3  East North Central Division   DV_ENC  1991       4     101.68    100.97  
4  East North Central Division   DV_ENC  1991       5     102.30    101.33  

First few rows of ZORI dataset:
   RegionID  SizeRank       RegionName RegionType StateName   2015-01-3

## 1.4 Dataset Cleaning

To clean the data set, we need to first melt the dataframe so that the Date columns are rows in order to make data manipulation easier. We will then combine the datasets in order to perform a more comprehensive analysis of the real estate market in New York.

In [3]:
ny = ['New York', 'NY']

#Filter each df based on states in the Northeast.
hpi_ny = hpi_df[hpi_df['place_name'].isin(ny)]
zori_ny = zori_df[zori_df['StateName'].isin(ny)]
zhvi_ny = zhvi_df[zhvi_df['StateName'].isin(ny)]
zordi_ny = zordi_df[zordi_df['StateName'].isin(ny)]

In [4]:
zori_ny.loc[:, 'SourceDataset'] = 'ZORI'
zhvi_ny.loc[:, 'SourceDataset'] = 'ZHVI'
zordi_ny.loc[:, 'SourceDataset'] = 'ZORDI'

/var/folders/9q/j_dp7nx93lbc_t5zbbshxpch0000gn/T/ipykernel_95009/1393036893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zori_ny.loc[:, 'SourceDataset'] = 'ZORI'
/var/folders/9q/j_dp7nx93lbc_t5zbbshxpch0000gn/T/ipykernel_95009/1393036893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zhvi_ny.loc[:, 'SourceDataset'] = 'ZHVI'
/var/folders/9q/j_dp7nx93lbc_t5zbbshxpch0000gn/T/ipykernel_95009/1393036893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [5]:
# Melt operation for zori_northeast
zori_ny = zori_ny.melt(id_vars=['SourceDataset', 'RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'],
                                     var_name='Date', value_name='Value')

# Melt operation for zhvi_northeast
zhvi_ny = zhvi_ny.melt(id_vars=['SourceDataset', 'RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'],
                                     var_name='Date', value_name='Value')

# Melt operation for zordi_northeast
zordi_ny = zordi_ny.melt(id_vars=['SourceDataset', 'RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'],
                                       var_name='Date', value_name='Value')

In [6]:
updated_df = pd.concat([zori_ny, zhvi_ny, zordi_ny], ignore_index=True)
updated_df

,SourceDataset,RegionID,SizeRank,RegionName,RegionType,StateName,Date,Value
0,ZORI,394913,1,"New York, NY",msa,NY,2015-01-31,2367.192976
1,ZORI,394425,50,"Buffalo, NY",msa,NY,2015-01-31,805.691732
2,ZORI,395031,52,"Rochester, NY",msa,NY,2015-01-31,947.847866
3,ZORI,394308,64,"Albany, NY",msa,NY,2015-01-31,1057.568717
4,ZORI,845159,86,"Poughkeepsie, NY",msa,NY,2015-01-31,1169.372109
...,...,...,...,...,...,...,...,...
10830,ZORDI,394634,579,"Gloversville, NY",msa,NY,2024-05-31,98.000000
10831,ZORDI,394326,607,"Amsterdam, NY",msa,NY,2024-05-31,138.000000
10832,ZORDI,394819,621,"Malone, NY",msa,NY,2024-05-31,NaN
10833,ZORDI,394504,629,"Cortland, NY",msa,NY,2024-05-31,31.000000


In [7]:
# Convert 'Date' column to datetime format
updated_df['Date'] = pd.to_datetime(updated_df['Date'])

In [8]:
updated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10835 entries, 0 to 10834
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SourceDataset  10835 non-null  object        
 1   RegionID       10835 non-null  int64         
 2   SizeRank       10835 non-null  int64         
 3   RegionName     10835 non-null  object        
 4   RegionType     10835 non-null  object        
 5   StateName      10835 non-null  object        
 6   Date           10835 non-null  datetime64[ns]
 7   Value          9825 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 677.3+ KB


### As noted in the info output above, there are 1010 null values in the merged dataframe. In order to fill in the null values, we have calculated the mean of the values by grouping each region by quarter and source. We then input the quarter mean into the Value column if the quarter mean was not null and the previous Value was. This left us with 953 null values which made up less than 10% of the population. As such, we removed those 953 null values leaving our dataframe with 9882 values. 

In [9]:
# Add 'Quarter' column based on 'Date'
updated_df['Quarter'] = updated_df['Date'].dt.to_period('Q')

# Group by 'RegionName', 'SourceDataset', and 'Quarter' to calculate mean
quarter_means = updated_df.groupby(['RegionName', 'SourceDataset', 'Quarter'])['Value'].mean().reset_index()

# Merge quarter_means back into the updated_df based on 'RegionName', 'SourceDataset', and 'Quarter'
updated_df = pd.merge(updated_df, quarter_means, on=['RegionName', 'SourceDataset', 'Quarter'], suffixes=('', '_QuarterMean'))

# Fill null values in 'Value' column with corresponding quarter means where available
updated_df['Value'].fillna(updated_df['Value_QuarterMean'], inplace=True)

# Drop the auxiliary 'Value_QuarterMean' column
updated_df.drop(columns='Value_QuarterMean', inplace=True)

# Display the updated_df
updated_df

,SourceDataset,RegionID,SizeRank,RegionName,RegionType,StateName,Date,Value,Quarter
0,ZORI,394913,1,"New York, NY",msa,NY,2015-01-31,2367.192976,2015Q1
1,ZORI,394913,1,"New York, NY",msa,NY,2015-02-28,2382.571737,2015Q1
2,ZORI,394913,1,"New York, NY",msa,NY,2015-03-31,2401.539081,2015Q1
3,ZORI,394425,50,"Buffalo, NY",msa,NY,2015-01-31,805.691732,2015Q1
4,ZORI,394425,50,"Buffalo, NY",msa,NY,2015-02-28,819.385346,2015Q1
...,...,...,...,...,...,...,...,...,...
10830,ZORDI,394819,621,"Malone, NY",msa,NY,2024-05-31,NaN,2024Q2
10831,ZORDI,394504,629,"Cortland, NY",msa,NY,2024-04-30,34.000000,2024Q2
10832,ZORDI,394504,629,"Cortland, NY",msa,NY,2024-05-31,31.000000,2024Q2
10833,ZORDI,395084,784,"Seneca Falls, NY",msa,NY,2024-04-30,65.000000,2024Q2


In [10]:
updated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10835 entries, 0 to 10834
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SourceDataset  10835 non-null  object        
 1   RegionID       10835 non-null  int64         
 2   SizeRank       10835 non-null  int64         
 3   RegionName     10835 non-null  object        
 4   RegionType     10835 non-null  object        
 5   StateName      10835 non-null  object        
 6   Date           10835 non-null  datetime64[ns]
 7   Value          9882 non-null   float64       
 8   Quarter        10835 non-null  period[Q-DEC] 
dtypes: datetime64[ns](1), float64(1), int64(2), object(4), period[Q-DEC](1)
memory usage: 762.0+ KB


In [11]:
df_1 = updated_df.dropna().copy()

In [12]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9882 entries, 0 to 10834
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SourceDataset  9882 non-null   object        
 1   RegionID       9882 non-null   int64         
 2   SizeRank       9882 non-null   int64         
 3   RegionName     9882 non-null   object        
 4   RegionType     9882 non-null   object        
 5   StateName      9882 non-null   object        
 6   Date           9882 non-null   datetime64[ns]
 7   Value          9882 non-null   float64       
 8   Quarter        9882 non-null   period[Q-DEC] 
dtypes: datetime64[ns](1), float64(1), int64(2), object(4), period[Q-DEC](1)
memory usage: 772.0+ KB
